In [1]:
import numpy as np 
import cv2
from skimage.feature import local_binary_pattern

In [2]:
import os
directory_list = list()
dir_path = "dataset"
for root, dirs, files in os.walk(dir_path, topdown=False):
    for name in dirs:
        directory_list.append((name))
for directory in directory_list:
    print(directory)

catagory_0
catagory_1
catagory_2
catagory_3
catagory_4
catagory_5
catagory_6
catagory_7


In [3]:
import glob

labels = 'labels.csv'
labelfile = open(labels,'w')
for i in range(len(directory_list)):
    readpath = (dir_path+'/' + directory_list[i]+'/*jpg')
    print("NN")
    print(readpath)
    images = glob.glob(readpath)
    for image in images:
        labelfile.write(image+','+str(i)+'\n')
labelfile.close()

NN
dataset/catagory_0/*jpg
NN
dataset/catagory_1/*jpg
NN
dataset/catagory_2/*jpg
NN
dataset/catagory_3/*jpg
NN
dataset/catagory_4/*jpg
NN
dataset/catagory_5/*jpg
NN
dataset/catagory_6/*jpg
NN
dataset/catagory_7/*jpg


In [4]:
import random
labels = 'labels.csv'
shuffled_labels = 'Shuffled_labels.csv'

labelfile = open(labels, "r")
lines = labelfile.readlines()
labelfile.close()
random.shuffle(lines)

shufflefile = open(shuffled_labels, "w")
shufflefile.writelines(lines)
shufflefile.close()

In [71]:
import cv2
import numpy as np
items = []
true_items = []
def add_dict(dic,val1,val2,val3):
    val=0
    for i in range(1,4):
        if(i==1):
            val=val1
        elif(i==2):
            val=val2
        elif(i==3):
            val=val3
        if (1000*i+val) not in dic.keys():
            dic[25000*i] = dic[25000*i] +1
        else:
            dic[i*1000+val] = dic[i*1000+val] + 1
    return dic
def add_patrn(dic,val1,val2,val3,val4):
    val=0
     
    for i in range(4,8):
        if(i==4):
            val=val1
        elif(i==5):
            val=val2
        elif(i==6):
            val=val3
        elif(i==7):
            val=val4
        if(1000*i+val) not in dic.keys():
            dic[25000*i]=dic[25000*i]+1
        else:
            dic[i*1000+val]=dic[i*1000+val]+1
    return dic
def lap(x,y,img):
    t=7
    sums=0
    var=0
    out=[]
    fx=[0,-1,-1,-1,0,1,1,1]
    fy=[1,1,0,-1,-1,-1,0,1]
    for i in range(0,8):
        sums+=int(img[x+fx[i]][y+fy[i]])
    mean=sums/8.0
    for i in range(0,8):
        var+=float((mean-int(img[x+fx[i]][y+fy[i]]))**2)
    var=var/8.0
    var=var**(0.5)                                               
    for i in range(0, 8):
        if(int(img[x][y])+t>=int(img[x+fx[i]][y+fy[i]])):
            if(int(img[x][y])-t<=int(img[x+fx[i]][y+fy[i]])):
                out.append(1)
            else:
                out.append(3)
        else:
             out.append(2)
    return out,var,mean
def pattern(var,mean,x,y,img):
    if(var>31):
        out=[]
        p=60
        q=88
        fx=[0,-1,-1,-1,0,1,1,1,0]
        fy=[1,1,0,-1,-1,-1,0,1,1]
        for i in range(0, 8):
            if(int(img[x+fx[i]][y+fy[i]])+q>=int(img[x+fx[i+1]][y+fy[i+1]])):
                if(int(img[x+fx[i]][y+fy[i]])+p<=int(img[x+fx[i+1]][y+fy[i+1]])):
                    out.append(1)
                else:
                    out.append(3)
            elif(int(img[x+fx[i]][y+fy[i]])-q<=int(img[x+fx[i+1]][y+fy[i+1]])):
                if(int(img[x+fx[i]][y+fy[i]])-p>=int(img[x+fx[i+1]][y+fy[i+1]])):
                    out.append(2)
                else:
                    out.append(4)
        return out,1
    else:
        return out,0
def local_ap_pattern(img,dic,q):
    try:
        for x in range(2, len(img)-2):
            for y in range(2, len(img[0])-2):
                print(x, y)
                value,ab,cd = lap(x,y,img)
                patrn,flag= pattern(ab,cd,x,y,img)
                #print("comes3", patrn)
                val1=0
                val2=0
                val3=0
                for i in range(0,len(value)):
                    if(value[i]==1):
                        val1+=2**(7-i)
                    elif(value[i]==2):
                        val2+=2**(7-i)
                    elif(value[i]==3):
                        val3+=2**(7-i)
                dic = add_dict(dic,val1,val2,val3)
                #print("comes3", patrn)
                val1=0
                val2=0
                val3=0
                val4=0
                if(flag==1):
                    for i in range(0,len(patrn)):
                        if(value[i]==1):
                            val1+=2**(7-i)
                        elif(value[i]==2):
                            val2+=2**(7-i)
                        elif(value[i]==3):
                            val3+=2**(7-i)
                        elif(value[i]==4):
                            val4+=2**(7-i)
                dic = add_patrn(dic,val1,val2,val3,val4)
            
            print(dic)
            print("Comes3") 
        return dic,1
    except TypeError as e:
        print ("type error "+str(j))
        return dic,0
    return dic,1
             

In [72]:
def uniform(pattern):
    pat= int(pattern)
    a=0
    b=0
    cnt=0
    for i in range(0,8):
        if( i==0 ):
            a= int(pattern/2**(7-i))
        else:
            b= int(pattern/2**(7-i))
            if(b!=a):
                cnt=cnt+1
                a=b
        pattern=pattern%2**(7-i)
    if(cnt<=2):
        return 1 #uniform hbe
    else:
        return 0
#items.append(0)
for i in range(0, 256):
    if uniform(i):
        items.append(1*1000+i)
        true_items.append(i)
        items.append(2*1000+i)
        items.append(3*1000+i)
        items.append(4*1000+i)
        items.append(5*1000+i)
        items.append(6*1000+i)
        items.append(7*1000+i)
items.append(25000)
items.append(50000)
items.append(25000*3)
items.append(25000*4)
items.append(25000*5)
items.append(25000*6)
items.append(25000*7)
true_items.append(25000)

In [73]:
import cv2
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
#from keras.utils import np_utils
from scipy.stats import itemfreq
import time
from functools import wraps

df = pd.read_csv(shuffled_labels,header=None)
rows = df.iterrows()

X_addrs = []
X_hist = []
Y_hist = []
#row = rows[0]
j = 0
a=-1
# uncomment below this for local tetra pattern
dic = {}
def create_dic():
    #for i in range(len(items)):
    for i in items:
        dic[i] = 0
    return 1
start_time = time.time()      
for row in rows:
    #print(row[1][1])
    create_dic()
    #print(row[1][0])
    img = cv2.imread(row[1][0])
    try:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        #print("comes")
        dic,a=local_ap_pattern(img,dic,j)
        print("comes2")
        new_x = []
        if(a==1):
            for i in dic.keys():
                new_x.append(dic[i])
            new_x = np.array(new_x)
            hist = new_x/np.sum(new_x)
            #print(hist)
            X_addrs.append(row[1][0])
            X_hist.append(hist)
            Y_hist.append(row[1][1])
            if(j%500==0):
                elapsed_time = time.time() - start_time
                start_time = time.time()
                print(str(j)+" done in " + str(elapsed_time))
    except Exception as e:
        print ("assertion error "+str(j))
    j = j + 1
    

2 2
assertion error 0
2 2
assertion error 1
2 2
assertion error 2
2 2
assertion error 3
2 2
assertion error 4
2 2
2 3
2 4
assertion error 5
2 2
assertion error 6
2 2
assertion error 7
2 2
assertion error 8
2 2
assertion error 9
2 2
assertion error 10
2 2
assertion error 11
2 2
assertion error 12
2 2
assertion error 13
2 2
2 3
2 4
2 5
assertion error 14
2 2
assertion error 15
2 2
assertion error 16
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
assertion error 17
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
assertion error 18
2 2
assertion error 19
2 2
assertion error 20
2 2
2 3
assertion error 21
2 2
assertion error 22
2 2
assertion error 23
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
assertion error 24
2 2
assertion error 25
2 2
2 3
assertion error 26
2 2
assertion error 27
2 2
assertion error 28
2 2
2 3
2 4
2 5
assertion error 29
2 2
assertion error 30
2 2
assertion error 31
2 2
2 3
assertion error 32
2 2
2 3
2 4
assertion error 33
2 2
2 3
2 4
assertion error 34
2 2
as

2 3
assertion error 327
2 2
2 3
2 4
assertion error 328
2 2
assertion error 329
2 2
assertion error 330
2 2
assertion error 331
2 2
assertion error 332
2 2
assertion error 333
2 2
assertion error 334
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
assertion error 335
2 2
assertion error 336
2 2
assertion error 337
2 2
assertion error 338
2 2
assertion error 339
2 2
assertion error 340
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
assertion error 341
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
assertion error 342
2 2
assertion error 343
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
2 28
2 29
2 30
2 31
2 32
2 33
2 34
2 35
2 36
2 37
2 38
2 39
2 40
2 41
2 42
2 43
2 44
2 45
2 46
2 47
2 48
2 49
2 50
2 51
2 52
2 53
2 54
2 55
2 56
2 57
2 58
2 59
2 60
2 61
2 62
2 63
2 64
2 65
2 66
2 67
2 68
2 69
2 70
2 71
2 72
2 73
2 74
2 75
2 76
2 77
2 78
2 79
2 80
2 81
2 82
2 83
2 84
2 85
2 86
2 87
2 8

2 5
2 6
assertion error 589
2 2
2 3
2 4
2 5
2 6
assertion error 590
2 2
assertion error 591
2 2
assertion error 592
2 2
2 3
2 4
2 5
assertion error 593
2 2
assertion error 594
2 2
assertion error 595
2 2
assertion error 596
2 2
assertion error 597
2 2
2 3
2 4
2 5
2 6
assertion error 598
2 2
2 3
assertion error 599
2 2
assertion error 600
2 2
assertion error 601
2 2
2 3
2 4
2 5
assertion error 602
2 2
2 3
assertion error 603
2 2
assertion error 604
2 2
assertion error 605
2 2
assertion error 606
2 2
assertion error 607
2 2
assertion error 608
2 2
assertion error 609
2 2
assertion error 610
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
assertion error 611
2 2
2 3
2 4
assertion error 612
2 2
assertion error 613
2 2
2 3
2 4
2 5
assertion error 614
2 2
assertion error 615
2 2
assertion error 616
2 2
2 3
assertion error 617
2 2
assertion error 618
2 2
assertion error 619
2 2
assertion error 620
2 2
assertion error 621
2 2
assertion error 622
2 2
assertion error 623
2 2
assertion e

2 2
assertion error 860
2 2
assertion error 861
2 2
assertion error 862
2 2
assertion error 863
2 2
2 3
2 4
2 5
2 6
assertion error 864
2 2
assertion error 865
2 2
assertion error 866
2 2
assertion error 867
2 2
assertion error 868
2 2
assertion error 869
2 2
assertion error 870
2 2
assertion error 871
2 2
assertion error 872
2 2
2 3
2 4
assertion error 873
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
assertion error 874
2 2
assertion error 875
2 2
assertion error 876
2 2
assertion error 877
2 2
assertion error 878
2 2
assertion error 879
2 2
2 3
2 4
assertion error 880
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
assertion error 881
2 2
2 3
2 4
2 5
2 6
2 7
2 8
assertion error 882
2 2
assertion error 883
2 2
assertion error 884
2 2
2 3
2 4
assertion error 885
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
assertion error 886
2 2
assertion error 887
2 2
assertion error 888
2 2
assertion error 889
2 2
assertion error 890
2 2
2 3
assertion error 891
2 2
assertion error 892
2 2
as

2 15
2 16
2 17
2 18
2 19
assertion error 1105
2 2
assertion error 1106
2 2
assertion error 1107
2 2
2 3
2 4
assertion error 1108
2 2
assertion error 1109
2 2
2 3
2 4
assertion error 1110
2 2
assertion error 1111
2 2
assertion error 1112
2 2
assertion error 1113
2 2
2 3
2 4
2 5
2 6
2 7
assertion error 1114
2 2
assertion error 1115
2 2
assertion error 1116
2 2
assertion error 1117
2 2
assertion error 1118
2 2
2 3
2 4
2 5
2 6
assertion error 1119
2 2
2 3
2 4
2 5
2 6
assertion error 1120
2 2
assertion error 1121
2 2
assertion error 1122
2 2
assertion error 1123
2 2
assertion error 1124
2 2
assertion error 1125
2 2
assertion error 1126
2 2
2 3
2 4
assertion error 1127
2 2
assertion error 1128
2 2
assertion error 1129
2 2
assertion error 1130
2 2
assertion error 1131
2 2
assertion error 1132
2 2
assertion error 1133
2 2
assertion error 1134
2 2
assertion error 1135
2 2
2 3
2 4
assertion error 1136
2 2
assertion error 1137
2 2
assertion error 1138
2 2
2 3
2 4
2 5
2 6
assertion error 1139
2 2


2 2
assertion error 1392
2 2
2 3
2 4
2 5
2 6
2 7
2 8
2 9
2 10
2 11
2 12
2 13
2 14
2 15
2 16
2 17
2 18
2 19
2 20
2 21
2 22
2 23
2 24
2 25
2 26
2 27
2 28
2 29
2 30
2 31
2 32
2 33
2 34
2 35
2 36
2 37
2 38
2 39
2 40
2 41
2 42
2 43
2 44
2 45
2 46
2 47
2 48
2 49
2 50
2 51
2 52
2 53
2 54
2 55
2 56
2 57
2 58
2 59
2 60
2 61
2 62
2 63
2 64
2 65
2 66
2 67
2 68
2 69
2 70
2 71
2 72
2 73
2 74
2 75
2 76
2 77
2 78
2 79
2 80
2 81
2 82
2 83
2 84
2 85
2 86
2 87
2 88
2 89
2 90
2 91
2 92
2 93
2 94
2 95
2 96
2 97
2 98
2 99
2 100
2 101
2 102
2 103
2 104
2 105
2 106
2 107
2 108
2 109
2 110
2 111
2 112
2 113
2 114
2 115
2 116
2 117
2 118
2 119
2 120
2 121
2 122
2 123
2 124
2 125
{1000: 3, 2000: 105, 3000: 0, 4000: 3, 5000: 105, 6000: 0, 7000: 124, 1001: 0, 2001: 1, 3001: 0, 4001: 0, 5001: 1, 6001: 0, 7001: 0, 1002: 1, 2002: 0, 3002: 0, 4002: 1, 5002: 0, 6002: 0, 7002: 0, 1003: 0, 2003: 1, 3003: 0, 4003: 0, 5003: 1, 6003: 0, 7003: 0, 1004: 0, 2004: 2, 3004: 0, 4004: 0, 5004: 2, 6004: 0, 7004: 0, 1006: 1, 2006: 

In [21]:
print(X_addrs[0])
print(X_hist[0])
print(Y_hist[0])
#print(X_hist)
X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)
print(len(X_addrs))
print(len(X_hist[50]))
print(X_hist.shape)
#print((Y_hist))

import pandas as pd 
df = pd.DataFrame(X_hist)
df.to_csv("tet_pattern_normalized_data_X_hist.csv",header=None)
df = pd.DataFrame(Y_hist)
df.to_csv("tet_pattern_normalized_label_Y_hist.csv",header=None)

IndexError: list index out of range

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv('tet_pattern_normalized_data_X_hist.csv', sep=',',header=None)
x = df
Y_hist= pd.read_csv('tet_pattern_normalized_label_Y_hist.csv', sep=',',header=None)
Y_hist = np.array(Y_hist)
Y_hist = Y_hist[:,1]
X_hist = []
x = np.array(x)
x = x[:,1:472]
print(len(x))
print(Y_hist.shape)

In [ ]:
for i in range (x.shape[0]):
    x1 = np.array(x[i][0:177])
    x1 = x1/np.sum(x1)
    x2 = np.array(x[i][177:413])
    x2 = x2/np.sum(x2)
    new_x = x1.tolist()+x2.tolist()
    #new_x = np.array(new_x)
    X_hist.append(new_x)

X_hist = np.array(X_hist)
Y_hist = np.array(Y_hist)
print(X_hist.shape, Y_hist.shape)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(np.array(X_hist), np.array(Y_hist), test_size=0.3)
X_train = np.array(X_train)
print(len(X_train))